# ENEL 645 Assignment 2

Tobin Eberle

Tom Wilson

Jeff Wheeler

Ryan Baker

### Ensure Virtual Environment is Activated

In [6]:
!which python

/Users/ryanb/miniforge3/envs/pytorch/bin/python


Environment is shown in `../envs/pytorch/..`, indicating that the `pytorch` virtual environment has been activated.

### Imports

In [7]:
import numpy as np
import os
import re
import time
import torch
import torchvision # type: ignore
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18 # type: ignore # pretrained model
from torchvision import models, transforms, datasets # type: ignore
from transformers import DistilBertModel, DistilBertTokenizer # type: ignore

from garbage_classifier_fusion import FusionNetwork, FusionDataSet,predict

### Define Constants
Note: Local copy of Test data was used to avoid TALC connection requirement

In [8]:
TEST_PATH = "./CVPR_2024_dataset_Test"
NUM_EPOCHS = 10
BATCH_SIZE = 24
NUM_FUSION_FEATURES = 100
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001
DROPOUT = 0.3
MAX_LENGTH = 200


### Variable & Model Instantiations

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
test_dataset = FusionDataSet(transform, tokenizer, TEST_PATH, MAX_LENGTH)
test_loader = DataLoader(test_dataset, shuffle = False)
imageModel = resnet18(weights='ResNet18_Weights.DEFAULT')
textModel = DistilBertModel.from_pretrained('distilbert-base-uncased')
model = FusionNetwork(imageModel, textModel, 4, NUM_FUSION_FEATURES, DROPOUT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load Weights
Including `map_location` is necessary as the notebook was run on a cpu device but trained on a CUDA device.

In [10]:
model.load_state_dict(torch.load("best_fusion_model.pth", map_location=device))


<All keys matched successfully>

In [11]:
test_predictions, test_classes = predict(model, test_loader, device)

Test Accuracy: 0.06%


In [12]:
set(test_classes)

{np.int64(1), np.int64(2), np.int64(3), np.int64(4)}

In [13]:
set(test_predictions)

{np.int64(0), np.int64(2), np.int64(3)}

In [14]:
# Count number of predicted true values
equal_count = 0
for pred, class_ in zip(test_predictions, test_classes):
    if pred == class_:
        equal_count += 1

In [15]:
equal_count

2